In [144]:
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np

In [145]:
# read images from local directory

im01 = np.array(Image.open('./RobertMixed03.jpg'))
im02 = np.array(Image.open('./smallstrelitzia.jpg'))
im03 = np.array(Image.open('./smallsunset.jpg'))
pixs01 = np.reshape(im01, (im01.shape[0]*im01.shape[1], 3))
pixs02 = np.reshape(im02, (im02.shape[0]*im02.shape[1], 3))
pixs03 = np.reshape(im03, (im03.shape[0]*im03.shape[1], 3))

In [146]:
# initialize parameters

    # Inputs: pixels: RGB image pixels
    #         segments: number of clusters
    #
    # Return: mu: cluster centers
    #         pi: cluster weights

def init_para(pixels, segments):
    
    kmeans = KMeans(n_clusters = segments, init = "random", random_state = 0).fit(pixels)
    mu = kmeans.cluster_centers_
    pi = np.ones((segments, 1))/segments
    
    return mu, pi

In [147]:
# E-step

    # Inputs: pixels: RGB image pixels
    #         mu: cluster centers
    #         pi: cluster weights    
    #
    # Return: w: weights that a data belongs to a cluster
    
def estimation(pixels, mu, pi):
    
    w = np.zeros((pixels.shape[0], mu.shape[0]))
    for x in range(pixels.shape[0]):
        
        pixel = np.array([pixels[x]]*mu.shape[0])
        diff = np.subtract(pixel, mu)
        square = np.multiply(diff, diff)
        min_idx = np.argmin(np.sum(square, axis = 1))
        dmin = square[min_idx]
        square = np.subtract(square, dmin)
        power = -1/2 * np.sum(square, axis=1)
        weight = np.multiply(np.exp(power), pi.T)
        w[x] = np.divide(weight, np.sum(weight))
        
    return w

In [148]:
# M-step

    # Inputs: pixels: RGB image pixels
    #         w: weights that a data belongs to a cluster  
    #
    # Return: mu: cluster centers
    #         pi: cluster weights  
    
def maximization(pixels, w):
    
    mu = np.zeros((w.shape[1], pixels.shape[1]))
    pi = np.zeros((w.shape[1], 1))
    for c in range(w.shape[1]):
        
        w_c = w.T[c]
        mu_num = np.matmul(pixels.T, w_c)
        mu_denom = np.sum(w_c)
        mu[c] = np.divide(mu_num, mu_denom)
        pi[c] = np.divide(mu_denom, pixels.shape[0])
        
    return mu, pi

In [149]:
# EM iteration

    # Inputs: it: iterating times
    #         pixels: RGB image pixels 
    #         mu: cluster centers
    #         pi: cluster weights  
    #
    # Return: mu: updated cluster centers after iterations
    #         pi: updated cluster weights after iterations

def EM(it, pixels, mu, pi):
    
    if it==0:
        
        return mu,pi
    
    else:
        
        w = estimation(pixels, mu, pi)
        mu, pi = maximization(pixels, w)
        
        return EM(it-1, pixels, mu, pi)

In [150]:
# image1 clustering

it = 10
segments = 10
mu0, pi0 = init_para(pixs01, segments)
mu, pi = EM(it, pixs01, mu0, pi0)

# TODO: plot the image using pixels closest to the center of clusters
# Your code here


In [152]:
print(mu)
print(pi)

[[ 11.72409518  42.98911235  23.00529445]
 [ 78.15518278 162.16218441 162.18803698]
 [100.758745   143.87469507  84.31850533]
 [ 60.96889802  99.50082165  42.72026448]
 [140.78219172 218.49049459 196.63742278]
 [200.55610242 251.65384917 231.31850604]
 [ 34.44835003 118.88114015 127.51713825]
 [150.33526502 189.46263054 118.84487247]
 [ 10.014494    76.16340445  79.80289943]
 [203.35118138 228.99082017 141.78535004]]
[[0.1634563 ]
 [0.07776837]
 [0.07435704]
 [0.07990646]
 [0.07755775]
 [0.07984125]
 [0.16509989]
 [0.10465653]
 [0.12803177]
 [0.04932463]]
